In [1]:
#Loading technique
from google.colab import files
uploaded =files.upload()

Saving news_tweets_sentiments.csv to news_tweets_sentiments.csv


In [9]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 1000, 'display.max_colwidth', 1000, 'display.max_rows',1000)

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Read Data

In [14]:
def read_data_header(filename,read_lines=3):
    # Stocks Data is in UTC

    #generate header from the file
    with open(filename) as myfile:
        head1 =[]
        for i in range (0,read_lines):
            head1.append([str.split(next(myfile).strip(),sep=',') for x in range(0,1)])

    line1 = head1[0][0]
    line2 = head1[1][0]

    header = [i.replace('^','')+' '+j for i, j in zip(line1, line2)]
    header = [item.replace('var','variance') for item in header]
    header = [item.replace(' ','') for item in header]
    header = [item.replace('=','') for item in header]

    return header

In [21]:
filename='/content/drive/MyDrive/ML3253-Project/news_tweets_sentiments.csv'
header = read_data_header(filename,2)
df= pd.read_csv(filename, sep=',',skiprows=2,names=header)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1133 entries, 0 to 1132
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  1133 non-null   object 
 1   combinedcount         1133 non-null   int64  
 2   combinedmean          1133 non-null   float64
 3   combinedvariance      1128 non-null   float64
 4   negativecount         1133 non-null   int64  
 5   negativemean          1133 non-null   float64
 6   negativevariance      1128 non-null   float64
 7   neutralcount          1133 non-null   int64  
 8   neutralmean           1133 non-null   float64
 9   neutralvariance       1128 non-null   float64
 10  positivecount         1133 non-null   int64  
 11  positivemean          1133 non-null   float64
 12  positivevariance      1128 non-null   float64
 13  CLFDirectionmean      1100 non-null   float64
 14  GSPCDirectionmean     1051 non-null   float64
 15  GSPTSEDirectionmean  

In [22]:
TARGET = "BTC-USDDirectionmean"
#columns_predictors = ['combined mean']
#columns_predictors = ['negativemean','neutralmean','positivemean']
columns_predictors = ['sentimentmean']

df = df[df[TARGET]!= 1] # remove data points when there is no price change
df[TARGET].replace(2,1, inplace=True)  # Replace label 2 with 1
df = df[columns_predictors+[TARGET]].dropna(how='any') # drop na values

X = df[columns_predictors]
y = df[TARGET]
y.unique()

array([1., 0.])

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10086)
print(f"X_train.shape: {X_train.shape}")
print(f"X_test.shape: {X_test.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"y_test.shape: {y_test.shape}")

X_train.shape: (849, 1)
X_test.shape: (284, 1)
y_train.shape: (849,)
y_test.shape: (284,)


In [57]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def printpp(text,colorc):
      return print(colorc + text + color.END)


In [68]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score,recall_score, f1_score

def classification_metric(y_test,y_pred):
    # Calculate accuracy
    printpp("accuracy_score %.4f" % accuracy_score(y_test, y_pred),color.GREEN)
    # precision
    printpp("precision_score %.4f"% precision_score(y_test,y_pred),color.GREEN)
    # recall
    printpp("recall_score %.4f"% recall_score(y_test,y_pred),color.GREEN)
    # F-1 score
    printpp("f1_score %.4f"% f1_score(y_test,y_pred),color.GREEN)
    # Confusion Matrix
    print(confusion_matrix(y_test,y_pred))
    # Classification Report
    #print(classification_report(y_test,y_pred))
    return None

def model_evaluation(model,X_train,y_train,X_test,y_test):
    model = model.fit(X_train,y_train)
    # training accuracy
    print('Evaluation for Train Data')
    classification_metric(y_train,model.predict(X_train))

    #testing accuracy
    print('Evaluation for Test Data')
    classification_metric(y_test,model.predict(X_test))
    return None

# Classification models are applied

- KNN, 
- Decision Trees, 
- RandomForest, 
- AdaBoost

In [70]:
# import xgboost
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import  LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
# from xgboost import XGBClassifier

from sklearn.model_selection import cross_val_score, GridSearchCV

# Set seed for reproducibility
SEED=1
# Instantiate lr
lr = LogisticRegression(solver='liblinear',random_state=SEED, verbose = 0)
# Instantiate knn
knn = KNeighborsClassifier(algorithm='kd_tree')
# Instantiate dt
dt = DecisionTreeClassifier(random_state=SEED)
#Instantiate rf
rf = RandomForestClassifier(random_state=SEED)
#Instantiate AdaBoost
AdaBC = AdaBoostClassifier(random_state=SEED)
# #Instantiate XGB
# XGBC = XGBClassifier()

# SVM Classifier Pipeline
polynomial_svm = Pipeline(steps=[
        ("poly_features", PolynomialFeatures()),
        ("svm_clf", SVC(random_state=18886))
    ])

# Define Polynomial Features
param_grid_polynomial_svm= {'poly_features__degree':[1,2,3],
            'svm_clf__C': [0.1, 1, 10, 100, 1000],
            'svm_clf__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
            'svm_clf__degree':[1,2,3],
            'svm_clf__kernel': ['linear','rbf','poly','sigmoid']}

param_grid_lr = {
    'penalty':['l1'],
    'C' : [0.01,0.1,0.9]
}

param_grid_knn = {
    'n_neighbors':np.arange(1,10,20),
    'leaf_size' : np.arange(1,50,5),
    'p':[1,2]
}

param_grid_dt = {
    'criterion':['gini','entropy'],
    'max_depth':[5,10,20]
}

param_grid_rf = {
    'criterion':['gini','entropy'],
    'max_depth':[5,10,20],
    'n_estimators':[10,20,100],
}

param_grid_adabc = {
    'n_estimators':[25,50,100],
    'learning_rate':[0.001, 0.01, 0.1]
}


# Define the list classifiers
classifiers = [('Logistic Regression', lr, param_grid_lr), ('K Nearest Neighbours', knn, param_grid_knn), 
               ('Decision Tree Classification ', dt, param_grid_dt), ('Random Forest' , rf,param_grid_rf), 
               ('AdaBoostClassifier', AdaBC, param_grid_adabc) , 
               ('Polynomial Features SVM', polynomial_svm, param_grid_polynomial_svm)
               ]


# Iterate over the pre-defined list of classifiers
for clf_name, clf, param_grid in classifiers:    
    x = clf_name+' Running'
    printpp(x,color.BLUE)
    # Define GridSearchCV
    grid = GridSearchCV(clf, param_grid, cv=3, n_jobs=1, verbose=1)
    
    # Fit clf to the training set
    grid.fit(X_train, y_train)

    # Best Model Parameters
    print(grid.best_params_)

    # Best Model Evaluation
    model_evaluation(grid.best_estimator_ ,X_train, y_train, X_test, y_test)
    

Logistic Regression Running
Fitting 3 folds for each of 3 candidates, totalling 9 fits
{'C': 0.01, 'penalty': 'l1'}
Evaluation for Train Data
accuracy_score 0.5041
precision_score 0.0000
recall_score 0.0000
f1_score 0.0000
[[428   0]
 [421   0]]
Evaluation for Test Data
accuracy_score 0.4718
precision_score 0.0000
recall_score 0.0000
f1_score 0.0000
[[134   0]
 [150   0]]
K Nearest Neighbours Running
Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'leaf_size': 6, 'n_neighbors': 1, 'p': 1}
Evaluation for Train Data
accuracy_score 0.5960
precision_score 0.5714
recall_score 0.7411
f1_score 0.6453
[[194 234]
 [109 312]]
Evaluation for Test Data
accuracy_score 0.5387
precision_score 0.5549
recall_score 0.6400
f1_score 0.5944
[[57 77]
 [54 96]]
Decision Tree Classification  Running
Fitting 3 folds for each of 6 candidates, totalling 18 fits
{'criterion': 'gini', 'max_depth': 20}
Evaluation for Train Data
accuracy_score 0.6419
precision_score 0.6364
recall_score 0.6485
f1_score 0.6424
[[272 156]
 [148 273]]
Evaluation for Test Data
accuracy_score 0.4789
precision_score 0.5070
recall_score 0.4800
f1_score 0.4932
[[64 70]
 [78 72]]
Random Forest Running
Fitting 3 folds for each of 18 candidates, totalling 54 fits
{'criterion': 'entropy', 'max_depth': 5, 'n_estimators': 10}
Evaluation for Train Data
accuracy_score 0.5866
precision_score 0.5994
recall_score 0.5012
f1_score 0.5459
[[287 141]
 [210 211]]
Evaluation for Test Data
accuracy_sc

# Redundant Code

In [ ]:
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

# Create a Random Forest Classifier with specified criterion
rf_class = RandomForestClassifier(criterion='entropy')

# Create the parameter grid
param_grid = {'max_depth': [2, 4, 8, 15], 'max_features': ['auto', 'sqrt']}

# Create a GridSearchCV object
grid_rf_class=GridSearchCV(
    estimator=rf_class,
    param_grid=param_grid,
    scoring='accuracy',
    n_jobs=4,
    cv=5,
    refit=True, return_train_score=True)
print(grid_rf_class)

GridSearchCV(cv=5, estimator=RandomForestClassifier(criterion='entropy'),
             n_jobs=4,
             param_grid={'max_depth': [2, 4, 8, 15],
                         'max_features': ['auto', 'sqrt']},
             return_train_score=True, scoring='accuracy')


In [ ]:
grid_rf_class.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=5, estimator=RandomForestClassifier(criterion='entropy'),
             n_jobs=4,
             param_grid={'max_depth': [2, 4, 8, 15],
                         'max_features': ['auto', 'sqrt']},
             return_train_score=True, scoring='accuracy')

In [ ]:


# Extract and print the column with a dictionary of hyperparameters used
column = cv_results_df.loc[:, ['params']]
print(column)

# Extract and print the row that had the best mean test score
best_row = cv_results_df[cv_results_df['rank_test_score'] == 1 ]
print(best_row)

NameError: ignored

In [ ]:
# Print out the ROC_AUC score from the best-performing square
best_score = grid_rf_class.best_score_
print(best_score)

# Create a variable from the row related to the best-performing square
cv_results_df = pd.DataFrame(grid_rf_class.cv_results_)
best_row = cv_results_df.loc[[grid_rf_class.best_index_]]
print(best_row)

# Get the n_estimators parameter from the best-performing square and print
best_n_estimators = grid_rf_class.best_params_
print(best_n_estimators)

In [ ]:
# See what type of object the best_estimator_ property is
print(type(grid_rf_class.best_estimator_))

# Create an array of predictions directly using the best_estimator_ property
predictions = grid_rf_class.best_estimator_.predict(X_test)

# Take a look to confirm it worked, this should be an array of 1's and 0's
print(predictions[0:5])

# Now create a confusion matrix 
print("Confusion Matrix \n", confusion_matrix(y_test, predictions))

# Get the ROC-AUC score
predictions_proba = grid_rf_class.best_estimator_.predict_proba(X_test)[:,1]
print("Accuracy Score \n", accuracy(y_test, predictions_proba))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Create the parameter grid
param_grid = {'max_depth': list(range(5,26)), 'max_features': ['auto' , 'sqrt']} 

# Create a random search object
random_rf_class = RandomizedSearchCV(
    estimator = RandomForestClassifier(n_estimators=80),
    param_distributions = param_grid, n_iter = 5,
    scoring='roc_auc', n_jobs=4, cv = 3, refit=True, return_train_score = True)

# Fit to the training data
random_rf_class.fit(X_train, y_train)
y_pred_rf = random_rf_class.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred) 
    # Confusion Matrix
c_matrix= confusion_matrix(y_test,y_pred)
    #classification reports
c_report = classification_report(y_test,y_pred)

# Print the values used for both hyperparameters
print(random_rf_class.cv_results_['param_max_depth'])
print(random_rf_class.cv_results_['param_max_features'])